<a href="https://colab.research.google.com/github/jonata-nuvem/codigoMare/blob/main/Bacteria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatório Curso Maré Bioinformática

# Montagem do driver

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Caminho para salvar dados do projeto

In [ ]:
path = '/content/drive/MyDrive/Bacteria'

# Instalando Conda - Conda é um gerenciador de pacotes

In [ ]:
!pip install -q condacolab

Instalando ferramenta [sra-tools] e baixando os dados brutos

In [ ]:
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...


In [ ]:
!conda install -c bioconda sra-tools --quiet

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - sra-tools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    curl-8.5.0                 |       hca28451_0          93 KB  conda-forge
    ncbi-vdb-3.1.1             |       h4ac6f70_0        10.7 MB  bioconda
    openssl-3.3.0              |       h4ab18f5_3         2.8 MB  conda-forge
    ossuuid-1.6.2              |    hf484d3e_1000          56 KB  conda-forge
    perl-5.22.0.1              |                0        15.1 MB  conda-forge
    perl-threaded-5.32.1       |       hdfd78af_1           5 KB  bioconda
    perl-uri-1.71              |       pl5

# Download dos dados brutos via sra

Baixar dados brutos de Cetobacterium sp. ZWU0022

In [ ]:
!prefetch -v SRR1573261


2024-05-24T13:37:16 prefetch.3.1.0: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2024-05-24T13:37:16 prefetch.3.1.0: 1) Downloading 'SRR1573261'...
2024-05-24T13:37:16 prefetch.3.1.0: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2024-05-24T13:37:16 prefetch.3.1.0:  Downloading via HTTPS...
2024-05-24T13:37:27 prefetch.3.1.0:  HTTPS download succeed
2024-05-24T13:37:28 prefetch.3.1.0:  'SRR1573261' is valid
2024-05-24T13:37:28 prefetch.3.1.0: 1) 'SRR1573261' was downloaded successfully
2024-05-24T13:37:28 prefetch.3.1.0: 'SRR1573261' has 0 unresolved dependencies


Criação do diretorio para dados brutos

In [ ]:
!mkdir $path/rawdata

Split dos dados brutos conversão em fastq "separar ou dividir os dados brutos de sequenciamento em unidades menores ou em diferentes arquivos"

In [ ]:

!fastq-dump --outdir $path/rawdata --split-files /content/SRR1573261/SRR1573261.sra

Read 922052 spots for /content/SRR1573261/SRR1573261.sra
Written 922052 spots for /content/SRR1573261/SRR1573261.sra


Comprimindo os arquivos

In [ ]:
!gzip $path/rawdata/*.fastq

Conferido tamanho dos arquivos

In [ ]:
!ls -lh  $path/rawdata

total 168M
-rw------- 1 root root 80M May 24 13:41 SRR1573261_1.fastq.gz
-rw------- 1 root root 89M May 24 13:41 SRR1573261_2.fastq.gz


# Análise primária, qualidade dos dados

In [ ]:
!conda install -c bioconda fastqc

Criando o diretório para a saída do fastqc

In [ ]:
!mkdir $path/fastqc

Exibir o help do software e a lista de opções

In [ ]:
!fastqc -h

Rodando o fastqc, gerando um report de métricas

In [ ]:
!fastqc $path/rawdata/*.gz --nogroup -o $path/fastqc

# Clipper & trimming (Remoção de sequencias e bases de baixa qualidade)

Instalando cutadapt

In [ ]:
!conda install -c bioconda cutadapt --quiet

criar pasta cutadapt

In [ ]:
!mkdir $path/cutadapt

Usando CUTADAPT EM comando em uma linha

In [ ]:
!cutadapt -q 30 -Q 30 -o $path/cutadapt/SRR1573261_1_2.fastq.gz -p $path/cutadapt/SRR1573261_2_2.fastq.gz $path/rawdata/SRR1573261_1.fastq.gz $path/rawdata/SRR1573261_2.fastq.gz

This is cutadapt 4.8 with Python 3.10.13
Command line parameters: -q 30 -Q 30 -o /content/drive/MyDrive/Bacteria/cutadapt/SRR1573261_1_2.fastq.gz -p /content/drive/MyDrive/Bacteria/cutadapt/SRR1573261_2_2.fastq.gz /content/drive/MyDrive/Bacteria/rawdata/SRR1573261_1.fastq.gz /content/drive/MyDrive/Bacteria/rawdata/SRR1573261_2.fastq.gz
Processing paired-end reads on 1 core ...
Done           00:00:27       922,052 reads @  29.8 µs/read;   2.02 M reads/minute
Finished in 27.458 s (29.779 µs/read; 2.01 M reads/minute).

=== Summary ===

Total read pairs processed:            922,052
Pairs written (passing filters):       922,052 (100.0%)

Total basepairs processed:   212,071,960 bp
  Read 1:   106,035,980 bp
  Read 2:   106,035,980 bp
Quality-trimmed:               9,047,837 bp (4.3%)
  Read 1:     3,622,750 bp
  Read 2:     5,425,087 bp
Total written (filtered):    203,024,123 bp (95.7%)
  Read 1:   102,413,230 bp
  Read 2:   100,610,893 bp


# Verificando a qualidade dos dados (Clipping)

criar pasta para o fastqc, gerando um report de métricas

In [ ]:
!mkdir $path/cutadapt/fastqc

In [ ]:
!fastqc $path/cutadapt/*.gz --nogroup -o $path/cutadapt/fastqc

application/gzip
Started analysis of SRR1573261_1_2.fastq.gz
application/gzip
Approx 5% complete for SRR1573261_1_2.fastq.gz
Approx 10% complete for SRR1573261_1_2.fastq.gz
Approx 15% complete for SRR1573261_1_2.fastq.gz
Approx 20% complete for SRR1573261_1_2.fastq.gz
Approx 25% complete for SRR1573261_1_2.fastq.gz
Approx 30% complete for SRR1573261_1_2.fastq.gz
Approx 35% complete for SRR1573261_1_2.fastq.gz
Approx 40% complete for SRR1573261_1_2.fastq.gz
Approx 45% complete for SRR1573261_1_2.fastq.gz
Approx 50% complete for SRR1573261_1_2.fastq.gz
Approx 55% complete for SRR1573261_1_2.fastq.gz
Approx 60% complete for SRR1573261_1_2.fastq.gz
Approx 65% complete for SRR1573261_1_2.fastq.gz
Approx 70% complete for SRR1573261_1_2.fastq.gz
Approx 75% complete for SRR1573261_1_2.fastq.gz
Approx 80% complete for SRR1573261_1_2.fastq.gz
Approx 85% complete for SRR1573261_1_2.fastq.gz
Approx 90% complete for SRR1573261_1_2.fastq.gz
Approx 95% complete for SRR1573261_1_2.fastq.gz
Analysis co

In [ ]:
!gunzip -c /content/drive/MyDrive/Bacteria/cutadapt/SRR1573261_1_2.fastq.gz | echo "read1" "$((`wc -l` / 4))"
!gunzip -c /content/drive/MyDrive/Bacteria/cutadapt/SRR1573261_2_2.fastq.gz | echo "read1" "$((`wc -l` / 4))"

read1 922052
read1 922052


# Instalando o spades (programa de montagem de genoma) e o bwa (ferramenta de alinhamento de sequencias de DNA)

In [ ]:
!conda install -c bioconda spades --quiet

# Montando um genoma de novo usando o spades

criando diretorio para Spades

In [ ]:
!mkdir $path/spades

Montando um genoma de novo usando o spades

In [ ]:
!spades.py -1 /content/drive/MyDrive/Bacteria/cutadapt/SRR1573261_1_2.fastq.gz -2 /content/drive/MyDrive/Bacteria/cutadapt/SRR1573261_2_2.fastq.gz -o $path/spades

Verifica as contigs

In [ ]:
!head -n 10 /content/drive/MyDrive/Bacteria/spades/contigs.fasta

>NODE_1_length_151163_cov_28.595945
AAATTATATCATTGCATTCTGCCAGAAAACCTACAATTTATTCTACCATTAACATATGAT
GTTTATGAAATCGAAAAATCTATAGAGGAATACTACTCTTCTATAAAAAAGTATTCTGAA
TTTTATTTAGGAGATACTTACTTAAAAGGAGTTTTTGATTTTTCTGAAATTTTATATATG
GATTACTGTAGAGTAAGTCGGAAAGTTAGATTTAATTTGATTAATAATGAGAGTTTTTTT
GTAAAACGAACATTCTCTGATTTAGATTTTTTAACAGTTACGTTTCCAAATTTTTATAAA
TTAGAAAGAGGGCTTATTATAAACACTGATTTAATCAAAATTTTAGATTATAAAGAGGAG
CGTATAATTTTTAAAGATAAATCAATCTTATATATCAGTAAAATTAAAATTAAAGAGCTT
GAAAGTATTCATAACTTTGAAAAAAATAGAGTATTCTTTTAAAGAAAAAGAACCTAAATA
ATTAAAATTTAGGTTCTTTTTTTATTAGATATTCTTTATTATATTCTTCAAAAATTCTTT


# Verificar o Numero de contigs

In [ ]:
!grep -c ">" /content/drive/MyDrive/Bacteria/spades/contigs.fasta

209


# Avaliando a qualidade da montagem

Instalando o quast

In [ ]:
!conda install -c bioconda quast --quiet

Criar diretório de saída

In [ ]:
!mkdir $path/quast

Analisando os dados

In [ ]:
!quast -o $path/quast $path/spades/contigs.fasta $path/spades/scaffolds.fasta

# Montagem por referência

# Configurando o genoma de refeência, dowload, index, dict, etc.

Para realizar a montagem do genoma por referência, devemos inicialmente obter e indexar a referência, para isso iremos até o NCBI e buscaremos em genomes o nome do organismo. Cetobacterium somerae

Configurando o ambiente

In [ ]:
!conda install -c bioconda picard samtools gatk4 bwa freebayes --quiet

In [ ]:
!mkdir $path/reference

Baixando genoma de referencia

In [ ]:
!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/022/430/525/GCA_022430525.1_ASM2243052v1/GCA_022430525.1_ASM2243052v1_genomic.fna.gz -P $path/reference

--2024-05-24 18:56:29--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/022/430/525/GCA_022430525.1_ASM2243052v1/GCA_022430525.1_ASM2243052v1_genomic.fna.gz
           => ‘/content/drive/MyDrive/Bacteria/reference/GCA_022430525.1_ASM2243052v1_genomic.fna.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.10, 2607:f220:41e:250::12, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCA/022/430/525/GCA_022430525.1_ASM2243052v1 ... done.
==> SIZE GCA_022430525.1_ASM2243052v1_genomic.fna.gz ... 838924
==> PASV ... done.    ==> RETR GCA_022430525.1_ASM2243052v1_genomic.fna.gz ... done.
Length: 838924 (819K) (unauthoritative)

GCA_022430525.1_ASM 100%[===================>] 819.26K  1.47MB/s    in 0.5s    

2024-05-24 18:56:31 (1.47 MB/s) - ‘/content/drive/MyDrive/Bacteria/reference/GCA_022430525.

In [ ]:
!gzip -d $path/reference/GCA_022430525.1_ASM2243052v1_genomic.fna.gz

Indexando o genoma de referencia

In [ ]:
!bwa index $path/reference/GCA_022430525.1_ASM2243052v1_genomic.fna

[bwa_index] Pack FASTA... 0.03 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 1.02 seconds elapse.
[bwa_index] Update BWT... 0.03 sec
[bwa_index] Pack forward-only FASTA... 0.02 sec
[bwa_index] Construct SA from BWT and Occ... 0.51 sec
[main] Version: 0.7.18-r1243-dirty
[main] CMD: bwa index /content/drive/MyDrive/Bacteria/reference/GCA_022430525.1_ASM2243052v1_genomic.fna
[main] Real time: 1.816 sec; CPU: 1.620 sec


In [ ]:
!samtools faidx $path/reference/GCA_022430525.1_ASM2243052v1_genomic.fna

In [ ]:
!picard CreateSequenceDictionary R=$path/reference/GCA_022430525.1_ASM2243052v1_genomic.fna O=$path/reference/GCA_022430525.1_ASM2243052v1_genomic.dict

## **Mapeado os read no genoma de referencia**

In [ ]:
!bwa mem -R "@RG\tID:SRR519926\tSM:SRR519926\tPL:MARE\tPU:unit1\tLB:lib1" $path/reference/GCA_022430525.1_ASM2243052v1_genomic.fna /content/drive/MyDrive/Bacteria/cutadapt/SRR1573261_1_2.fastq.gz /content/drive/MyDrive/Bacteria/cutadapt/SRR1573261_2_2.fastq.gz > $path/reference/SRR1573261.sam

In [ ]:
!samtools sort $path/reference/SRR1573261.sam > $path/reference/SRR1573261.bam

In [ ]:
!samtools index $path/reference/SRR1573261.bam

In [ ]:
!samtools view $path/reference/SRR1573261.bam | wc -l

1844604


# Usar GATK (Genome Analysis Toolkit) HaplotypeCaller [mais detalhado que o Freebayes]

In [ ]:
!gatk --java-options "-Xmx10g" HaplotypeCaller -R /content/drive/MyDrive/Bacteria/reference/GCA_022430525.1_ASM2243052v1_genomic.fna -I /content/drive/MyDrive/Bacteria/reference/SRR1573261.bam -O /content/drive/MyDrive/Bacteria/reference/SRR1573261.vcf --sample-ploidy 1

In [ ]:
!mkdir $path/reference2

In [ ]:
!samtools quickcheck /content/drive/MyDrive/Bacteria/reference/SRR1573261.bam

In [ ]:
!samtools faidx /content/drive/MyDrive/Bacteria/reference/GCA_022430525.1_ASM2243052v1_genomic.fna

Freebayes -Opção de análise de variante, porém, não foi necessário aqui. no entanto, os arquivos gerados estão na pasta /reference2

In [ ]:
#!freebayes -f /content/drive/MyDrive/Bacteria/reference/GCA_022430525.1_ASM2243052v1_genomic.fna /content/drive/MyDrive/Bacteria/reference/SRR1573261.bam > /content/drive/MyDrive/Bacteria/reference2/SRR1573261.freebayes.vcf


Checando as variantes encontradas

In [ ]:
# @title Texto de título padrão
!grep -v "##" /content/drive/MyDrive/Bacteria/reference/SRR1573261.vcf | head -n 10

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SRR519926
CP092519.1	21	.	A	G	1115.04	.	AC=1;AF=1.00;AN=1;DP=25;FS=0.000;MLEAC=1;MLEAF=1.00;MQ=54.46;QD=25.36;SOR=2.584	GT:AD:DP:GQ:PL	1:0,25:25:99:1125,0
CP092519.1	29	.	A	G	1520.04	.	AC=1;AF=1.00;AN=1;DP=34;FS=0.000;MLEAC=1;MLEAF=1.00;MQ=55.98;QD=28.73;SOR=3.258	GT:AD:DP:GQ:PL	1:0,34:34:99:1530,0
CP092519.1	30	.	T	A	1520.04	.	AC=1;AF=1.00;AN=1;DP=34;FS=0.000;MLEAC=1;MLEAF=1.00;MQ=55.98;QD=30.97;SOR=3.258	GT:AD:DP:GQ:PL	1:0,34:34:99:1530,0
CP092519.1	34	.	A	G	1610.04	.	AC=1;AF=1.00;AN=1;DP=36;FS=0.000;MLEAC=1;MLEAF=1.00;MQ=56.21;QD=27.24;SOR=3.383	GT:AD:DP:GQ:PL	1:0,36:36:99:1620,0
CP092519.1	36	.	C	T	1610.04	.	AC=1;AF=1.00;AN=1;DP=36;FS=0.000;MLEAC=1;MLEAF=1.00;MQ=56.21;QD=28.20;SOR=3.383	GT:AD:DP:GQ:PL	1:0,36:36:99:1620,0
CP092519.1	39	.	A	T	1610.04	.	AC=1;AF=1.00;AN=1;DP=36;FS=0.000;MLEAC=1;MLEAF=1.00;MQ=56.21;QD=25.00;SOR=3.383	GT:AD:DP:GQ:PL	1:0,36:36:99:1620,0
CP092519.1	42	.	A	G	1641.04	.	AC=1;AF=1.00;AN=1;DP=53;FS=0.000;MLEAC=1;MLE